In [2]:
import tkinter as tk
from tkinter import simpledialog, messagebox, filedialog
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
import cv2
import numpy as np
import os
import pandas as pd
from datetime import datetime
from PIL import Image, ImageTk

# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True, min_face_size=40)

# Initialize InceptionResnetV1 for face recognition
facenet = InceptionResnetV1(pretrained='vggface2').eval()

# Paths for storing data
database_path = r"D:\face_detection\face_database"
attendance_file = os.path.join(database_path, "attendance.xlsx")

# Admin credentials (for demonstration purposes, store securely in a real app)
admin_username = "admin"
admin_password = "admin123"

# Work start time for late arrival tracking
work_start_time = "09:00:00"

# Ensure database path exists
if not os.path.exists(database_path):
    os.makedirs(database_path)

# Ensure attendance file exists
if not os.path.exists(attendance_file):
    df = pd.DataFrame(columns=["User ID", "Date", "Time", "Status"])
    df.to_excel(attendance_file, index=False)

class AttendanceSystem:
    def __init__(self, master):
        self.master = master
        self.master.title("Company Attendance System")

        # Video Feed Label
        self.video_label = tk.Label(master)
        self.video_label.pack()

        # Buttons
        self.register_button = tk.Button(master, text="Register User", command=self.register_user)
        self.register_button.pack(pady=10)

        self.login_button = tk.Button(master, text="Mark Attendance", command=self.recognize_user)
        self.login_button.pack(pady=10)

        self.admin_login_button = tk.Button(master, text="Admin Panel", command=self.admin_login)
        self.admin_login_button.pack(pady=10)

        # Initialize video capture
        self.cap = cv2.VideoCapture(0)
        self.update_frame()

    def update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
        self.video_label.after(10, self.update_frame)

    def register_user(self):
        user_id = simpledialog.askstring("Input", "Enter User ID for registration:")
        if not user_id:
            return

        ret, frame = self.cap.read()
        if not ret:
            messagebox.showwarning("Registration", "Failed to capture frame.")
            return

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = mtcnn(frame_rgb)

        if faces is None or len(faces) == 0:
            messagebox.showwarning("Registration", "No face detected.")
            return

        try:
            face_tensor = faces[0]
            embedding = facenet(face_tensor.unsqueeze(0)).detach().numpy().flatten()
            embedding /= np.linalg.norm(embedding)  # Normalize embedding
            
            cv2.imwrite(os.path.join(database_path, f"{user_id}.jpg"), frame)
            np.save(os.path.join(database_path, f"{user_id}.npy"), embedding)
            messagebox.showinfo("Registration", f"User {user_id} registered successfully!")
            
        except Exception as e:
            messagebox.showerror("Error", f"Registration failed: {str(e)}")

    def recognize_user(self):
        ret, frame = self.cap.read()
        if not ret:
            messagebox.showwarning("Recognition", "Failed to capture frame.")
            return
    
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = mtcnn(frame_rgb)
    
        if faces is None or len(faces) == 0:
            messagebox.showwarning("Recognition", "No face detected.")
            return
    
        if len(faces) > 1:
            messagebox.showerror("Recognition", "Multiple faces detected! Please ensure only one person is in the frame.")
            return
    
        try:
            face_tensor = faces[0]  # Only one face allowed
            live_embedding = facenet(face_tensor.unsqueeze(0)).detach().numpy().flatten()
            live_embedding /= np.linalg.norm(live_embedding)  # Normalize
    
            best_match = None
            best_similarity = -1
    
            for filename in os.listdir(database_path):
                if filename.endswith(".npy"):
                    user_id = filename[:-4]
                    stored_embedding = np.load(os.path.join(database_path, filename))
                    stored_embedding /= np.linalg.norm(stored_embedding)
    
                    similarity = np.dot(live_embedding, stored_embedding)
    
                    if similarity > best_similarity:
                        best_similarity = similarity
                        best_match = user_id
    
            if best_match and best_similarity > 0.6:
                self.mark_attendance(best_match)
                messagebox.showinfo("Recognition", f"User {best_match} recognized! (Score: {best_similarity:.2f})")
            else:
                messagebox.showwarning("Recognition", "No matching face found.")
    
        except Exception as e:
            messagebox.showerror("Error", f"Recognition failed: {str(e)}")

    def mark_attendance(self, user_id):
        now = datetime.now()
        date_today = now.strftime("%Y-%m-%d")
        time_now = now.strftime("%H:%M:%S")

        df = pd.read_excel(attendance_file)

        # Check if the user has already marked attendance today
        if ((df["User ID"] == user_id) & (df["Date"] == date_today)).any():
            messagebox.showinfo("Attendance", f"Attendance already marked for {user_id} today.")
            return

        # Determine status (On-time or Late)
        status = "On-time" if time_now <= work_start_time else "Late"

        # Append new attendance record
        new_record = pd.DataFrame({"User ID": [user_id], "Date": [date_today], "Time": [time_now], "Status": [status]})
        df = pd.concat([df, new_record], ignore_index=True)
        df.to_excel(attendance_file, index=False)

        messagebox.showinfo("Attendance", f"Attendance marked for {user_id} at {time_now}. Status: {status}")

    def admin_login(self):
        username = simpledialog.askstring("Admin Login", "Enter Admin Username:")
        password = simpledialog.askstring("Admin Login", "Enter Admin Password:", show="*")

        if username == admin_username and password == admin_password:
            self.admin_panel()
        else:
            messagebox.showerror("Login Failed", "Incorrect credentials!")

    def admin_panel(self):
        option = simpledialog.askstring("Admin Panel", "Enter 1 to View Attendance, 2 to Export Report:")

        if option == "1":
            self.view_attendance()
        elif option == "2":
            self.export_attendance()

    def view_attendance(self):
        df = pd.read_excel(attendance_file)
        records = df.to_string(index=False)
        messagebox.showinfo("Attendance Records", records)

    def export_attendance(self):
        export_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel Files", "*.xlsx")])
        if export_path:
            df = pd.read_excel(attendance_file)
            df.to_excel(export_path, index=False)
            messagebox.showinfo("Export", "Attendance report exported successfully!")

    def on_closing(self):
        if self.cap.isOpened():
            self.cap.release()
        self.master.destroy()

root = tk.Tk()
app = AttendanceSystem(root)
root.protocol("WM_DELETE_WINDOW", app.on_closing)
root.mainloop()